In [1]:
import os, sys
project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

attention_dir = os.path.join(project_dir, 'modules/AttentionMap')
if attention_dir not in sys.path:
    sys.path.append(attention_dir)

sparse_dir = os.path.join(project_dir, 'modules/Sparse')
if sparse_dir not in sys.path:
    sys.path.append(sparse_dir) 

import config
from derma.dataset import Derma
from derma.architecture import InvertedResidual

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models import MobileNetV2
from torchvision import transforms

torch.cuda.is_available()

True

# Problem parameters

In [2]:
Test = 'T3'
DB_used = 'DB1_lr1-4'

dataset_dir = os.path.join(config.DATASET_DIR,'HAM10000_balanced_000')
#dataset_dir = os.path.join(config.DATASET_DIR,'ISIC2019_balanced_000_256')

CIFAR_dir = os.path.join(config.RESULT_DIR,'weights',Test,'features.pth')
log_dir = os.path.join(config.RESULT_DIR,'log',Test,DB_used) 
model_dir = os.path.join(config.RESULT_DIR,'weights',Test,DB_used,'model.pth')

CoordAtt = True

inverted_residual_setting_v0 = [
        # t, c, n, s
        [1, 16, 1, 1],
        [6, 24, 2, 2],
        [6, 32, 3, 2],
        [6, 64, 4, 2],
        [6, 96, 3, 1],
        [6, 160, 3, 2],
        [6, 320, 1, 1],
    ] #ORIGINAL

inverted_residual_setting_vT3 = [
        # t, c, n, s
        [1, 16, 1, 1],
        [4, 24, 1, 2],
        [4, 32, 1, 2],
        [4, 64, 1, 2],
        [4, 96, 1, 1],
        [4, 160, 1, 2],
        [4, 320, 1, 1],
    ]

inverted_residual_setting = inverted_residual_setting_vT3

In [3]:
# OTHER PARAMS

labels = [0, 1]
split_ratio = [0.8,0.1,0.1] # train, val, test
Weighted_sampling = True
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip()
])
lr = 1e-4
n_epoch = 20
batch_size = 40
criterion = torch.nn.CrossEntropyLoss()

# Dataset

In [4]:
# Derma dataset 
dataset = Derma(dataset_dir,labels=labels,transform=transform)

torch.manual_seed(config.RANDOM_SEED)

# Train-test splittings
train_set, val_set, test_set = dataset.split_rand(split_ratio=split_ratio)

In [5]:
# Weighted sampling
if Weighted_sampling:
    from derma.dataset import get_samples_weight
    train_sampler, train_weights = get_samples_weight(train_set)
    train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=0, sampler=train_sampler, shuffle=False)
    val_sampler, val_weights = get_samples_weight(val_set)
    val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=0, sampler=val_sampler, shuffle=False)
    test_sampler, test_weights = get_samples_weight(test_set)
    test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=0, sampler=test_sampler, shuffle=False)
else:
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=0)


Samples per class: [885 895]
Weight per class: [0.00112994 0.00111732]
Samples per class: [106 116]
Weight per class: [0.00943396 0.00862069]
Samples per class: [122 102]
Weight per class: [0.00819672 0.00980392]


# Model

In [6]:
if CoordAtt:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting, block=InvertedResidual)
else:
    model = MobileNetV2(num_classes=len(labels), inverted_residual_setting=inverted_residual_setting) # standard MobileNetV2

In [7]:
# LOADIONG CIFAR WEIGHTS
features = torch.load(CIFAR_dir)
model.features.load_state_dict(features)

<All keys matched successfully>

# Training

In [8]:
from derma.utils import train
from torch.utils.tensorboard import SummaryWriter

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
tb_writer = SummaryWriter(log_dir=log_dir)

train(model, [train_loader,val_loader], optimizer, criterion, n_epoch, tb_writer)

100%|██████████| 20/20 [05:32<00:00, 16.65s/epoch, acc=0.8750, tls=0.2967]


0.3004938215017319

In [9]:
torch.save(model.features.state_dict(),model_dir)


# Test

In [10]:
#model_dir = 
if torch.cuda.is_available():
    device = torch.device('cuda')
    model.features.load_state_dict(torch.load(model_dir))
    model.to(device)
else:
    model.features.load_state_dict(torch.load(model_dir),map_location=torch.device('cpu'))

In [11]:
from derma.experiment import test_experiment
metrics_df = test_experiment(model,test_loader)
metrics_df.head()

,Sensitivity,Specificity,Precission,Recall,Accuracy
0,0.909091,0.769231,0.769231,0.909091,0.833333
